In [1]:
from tia_util import remove_diag_reshape

In [2]:
import pickle as pk
import torch 
import numpy as np
import networkx as nx
import os
import pandas as pd
from attack_models_sc4 import contrastive_normalization_np_withpower

In [8]:

from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.neural_network import MLPClassifier

In [14]:
cur_dir = os.getcwd()
metrics_root = f"{cur_dir}\\saved_metrics\\"
topology_path = f"{cur_dir}/topologies/"

In [30]:
TOPOLOGY = ["star", "ring","ER_0.3", "ER_0.5", "ER_0.7",'Abilene', 'brain', 'GÉANT', 'synth50', 'rf1755', 'rf3967', 'atlanta', 'brain',  'cost266', 'dfn-bwin', 'dfn-gwin', 'di-yuan', 'france',  'germany50', 'giul39', 'india35', 'janos-us', 'janos-us-ca', 'newyork', 'nobel-eu', 'nobel-germany', 'nobel-us', 'norway', 'pdh', 'pioro40', 'polska', 'sun', 'ta1', 'ta2', 'zib54']
NUM_CLIENTS = [10, 20,30,12,22,50,79,87, 15, 161, 37, 11, 25, 35, 26, 39, 16, 28, 17, 14,  40,  27, 24, 65, 54]
TOPOLOGY = ["star", "ring","ER_0.3", "ER_0.5", "ER_0.7"]
# NUM_CLIENTS = [10, 20,30]
# DATASET = ["Cifar10no", "Cifar10", "Mnist","FMnist", "imagenet10", "pcam", "svhn"]
DATASET = ["Cifar10"]
MODEL = ["mlp", "mobile","resnet","pf"]
# MODEL = ["mlp", "mobile", "resnet"]
IID = [1]
MAX_EPOCHS = [3]
ALPHA = 0.1
SEED = [42]
all_metrics_sc = os.listdir(metrics_root)

In [24]:
import copy

In [33]:
all_res = []
for dataset in DATASET:
        for model_name in MODEL:
            # dataset and model setting
            for topo in TOPOLOGY:
               for iid in IID:
                    for seed in SEED:
                        for max_epoch in MAX_EPOCHS:
                            for num in NUM_CLIENTS:
                                ROUND =  num
                                toponame_file = f"{topology_path}/{num}_{topo}.pk"
                                if os.path.exists(toponame_file):  
                                    with open(toponame_file, "rb") as f:
                                        G = pk.load(f)
                                        density = nx.density(G)
                                        label = nx.adjacency_matrix(G).todense()
                                else:
                                    continue
                                
                                scenario_name = dataset + "_" + model_name + "_" + topo + "_" + str(iid) + "_" + str(ALPHA) + "_" + str(seed) + "_" + str(max_epoch) + "_" + str(num)
                                if scenario_name in all_metrics_sc:
                                    all_metrics = {}
                                    # for approach in ['_cosine_metric', '_curvature_divergence', '_entropy_metric', '_euclidean_metric', '_jacobian_metric', '_loss_metric']:
                                    try:
                                        for approach in ['_cosine_metric', '_loss_metric']:                                                                          
                                            metric_path = os.path.join(metrics_root,scenario_name, f'{approach}.csv')                                        
                                            metrics = pd.read_csv(metric_path, index_col=0)
                                            all_metrics[approach] = metrics
                                    except:
                                        continue
                                            
                                    y = remove_diag_reshape(label).flatten()
                                    cosval = np.copy(all_metrics['_cosine_metric'])
                                    cosval = contrastive_normalization_np_withpower(cosval, eta=3)
                                    X_cos = remove_diag_reshape(cosval).flatten()
                                    # X = X_cos.reshape((len(X_cos),1))

                                    lossval = np.copy(all_metrics['_cosine_metric'])
                                    lossval = contrastive_normalization_np_withpower(lossval, eta=3)
                                    X_loss= remove_diag_reshape(lossval).flatten()
                                    # # X_loss = X_loss.reshape((len(X_loss),1))

                                    X = np.stack([X_cos, X_loss], axis=1)
                                    X = X.reshape((len(X),2))
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
                                    
                                    # res = {}
                                    # model = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
                                    # model.fit(X_train, y_train)
                                    # y_pred = model.predict(X_test)
                                    # f1 = f1_score(y_test, y_pred, average='micro')
                                    # res['value'] = f1
                                    # res['alg'] = "LogisticRegression"
                                    # res['dataset'] = dataset
                                    # res['topo'] = topo                                        
                                    # res['iid'] = iid
                                    # res['max_epoch'] = max_epoch
                                    # res['num'] = num
                                    # res['density'] = density
                                    # res['approach'] = approach
                                    # all_res.append(res)
                                    
                                    # res = {}
                                    # model = RandomForestClassifier(class_weight='balanced', random_state=42)
                                    # model.fit(X_train, y_train)
                                    # y_pred = model.predict(X_test)
                                    # f1 = f1_score(y_test, y_pred, average='micro')
                                    # res['value'] = f1
                                    # res['alg'] = "RandomForestClassifier"
                                    # res['dataset'] = dataset
                                    # res['topo'] = topo                                        
                                    # res['iid'] = iid
                                    # res['max_epoch'] = max_epoch
                                    # res['num'] = num
                                    # res['density'] = density
                                    # res['approach'] = approach
                                    # all_res.append(res)
                                    
                                    # res = {}
                                    # model = SVC(class_weight='balanced', random_state=42)
                                    # model.fit(X_train, y_train)
                                    # y_pred = model.predict(X_test)
                                    # f1 = f1_score(y_test, y_pred, average='micro')
                                    # res = {}
                                    # res['value'] = f1
                                    # res['alg'] = "SVC"
                                    # res['dataset'] = dataset
                                    # res['topo'] = topo                                        
                                    # res['iid'] = iid
                                    # res['max_epoch'] = max_epoch
                                    # res['num'] = num
                                    # res['density'] = density
                                    # res['approach'] = approach
                                    # all_res.append(res)
                                    
                                    model = MLPClassifier(random_state=42, max_iter=500, solver='adam')
                                    model.fit(X_train, y_train)
                                    y_pred = model.predict(X_test)
                                    f1 = f1_score(y_test, y_pred, average='micro')
                                    print("EDGEPRE Test F1:", f1)
                                    res = {}
                                    res['value'] = f1
                                    res['alg'] = "EDGEPRE"
                                    res['dataset'] = dataset
                                    res['topo'] = topo                                        
                                    res['iid'] = iid
                                    res['max_epoch'] = max_epoch
                                    res['num'] = num
                                    res['density'] = density
                                    res['approach'] = approach
                                    all_res.append(res)                                                                   
                                                            

j:\TIA\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


EDGEPRE Test F1: 1.0
EDGEPRE Test F1: 0.9824561403508771
EDGEPRE Test F1: 0.9846743295019157
EDGEPRE Test F1: 1.0
EDGEPRE Test F1: 1.0
EDGEPRE Test F1: 1.0
EDGEPRE Test F1: 0.9629629629629629
EDGEPRE Test F1: 0.8070175438596491
EDGEPRE Test F1: 0.8505747126436781
EDGEPRE Test F1: 0.9259259259259259
EDGEPRE Test F1: 0.6842105263157895
EDGEPRE Test F1: 0.7241379310344828
EDGEPRE Test F1: 0.9259259259259259
EDGEPRE Test F1: 0.6929824561403509
EDGEPRE Test F1: 0.6973180076628352


In [34]:
res_df = pd.DataFrame(all_res)
res_df.to_csv('sc1_cfaug.csv')